# How to run cells:

Click the run button above in the toolbar (▶), or press <kbd>Shift</kbd> + <kbd>Enter</kbd>

## Cell 1: Find spreadsheets in the project. 

Before running this cell, make sure to upload your spreadsheet into the file explorer on the left side (icon looks like ![Upload Icon](%2BPC9nPjwvc3ZnPg%3D%3D)).

Once you've uploaded the spreadsheet, it will show up in the little side file listing. Running this cell will print out the name of the spreadsheet if everything works correctly.

In [1]:
# Install openpyxl so we can read and save excel spreadsheets in pandas.
print("Starting!")
import micropip
await micropip.install('ipywidgets')
%pip install openpyxl
# Import needed libraries. Key one here is pandas, which allows us to do operate on "DataFrames" which are spreadsheet objects.
from pathlib import Path
import pandas as pd

# Iterate files in this directory, if it ends with ".xlsx" keep it. We also filter out the spreadsheet name we save at the end.
sheet_paths = [p for p in Path().iterdir() if(p.suffix == ".xlsx" and p.name != "duplicates.xlsx")]

# Print the set of spreadsheets a
print("Spreadsheets:", ", ".join(str(p) for p in sheet_paths))

Starting!


ModuleNotFoundError: No module named 'micropip'

# Cell 2: Load the Spreadsheet.

This loads the first spreadsheet in the above list into a pandas DataFrame object. This object acts much like a spreadsheet, and allows editing the sheet. You could replace `sheet_paths[0]` below with just the name of the speadsheet you want to load in quotes (such as `"MySpreadsheet.xlsx"`).

In [2]:
# Read the spreadsheet into a panda's DataFrame object. Here we assign it to a variable, called first_sheet, but you could call it anything.
first_sheet = pd.read_excel(sheet_paths[0], header=None)

# Cell 3: Display the Spreadsheet

In jupyter, you can display the spreadsheet by simply just entering it's name at the end of the cell.

In [3]:
# Display the spreadsheet.
first_sheet

,0,1,2
0,A,B,C
1,E,F,G
2,H,I,J
3,A,B,C
4,C,A,D
5,D,O,G


# Cell 4: Grab Only Duplicate Columns

Pandas dataframe objects provide a method called `duplicated` that returns a boolean mask, where there is a 1 (or True) for every location that is a duplicate, and a 0 (or False) everywhere else. By indexing the spreadsheet (the square brackets below) with this mask, we get every single row that has duplicates. This operation generates a new dataframe, which we store in a variable called `duplicates`.

In [4]:
# Grab every duplicated row of the spreadsheet (keep=False tells it to keep all the duplicates, not just the first or last).
duplicates = first_sheet[first_sheet.duplicated(keep=False)]
# Display the new table, which only has rows with duplicates.
duplicates

,0,1,2
0,A,B,C
3,A,B,C


# Cell 5: Save the Dataframe back to an Excel Spreadsheet

This saves the duplicates only dataframe object back to a spreadsheet, using `to_excel` method. It accepts a single argument, the name of the spreadsheet. You can change the name by modifying the text in quotes.

In [5]:
# Save the duplicates to a new file...
duplicates.to_excel("duplicates.xlsx", header=None, index=None)

Once done, you'll see a `duplicates.xlsx` in the file explorer on the left. Right click on it and click `Download` to download the file.